In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Project_data = pd.read_excel('data.xlsx', header=0)

## Understanding the data
checking the Project_data columns and number of missing values


In [ ]:
Project_data.info()

The columns which do not have influence on the type of risks are dropped. This includes the ID link, the phase of the execution (because we are only predicting at the start anyway), and the names of the people on the project. The country is also dropped since the focus is on Dutch projects.

The project number and name are not dropped because this will make it easier to identify the past project that is most similar to a new project. However, these two columns will not be used as features for the algorithm.

In [ ]:
Project_data = Project_data.drop(columns=['SharePoint', 'Fase', 'Land', 'Directeur projecten', 'Bedrijfsleider', 'Project Manager', 'Tender Manager'])
Project_data.head()

In [ ]:
#Project_data.iloc[:,2:].to_csv('Info sheets\\Project_info.csv', index=False)

Changing column types

In [ ]:
Project_data['Locatie'] = Project_data['Locatie'].astype('category')
Project_data['Businessline'] = Project_data['Businessline'].astype('category')
Project_data['Opdrachtgever'] = Project_data['Opdrachtgever'].astype('category')
Project_data['Type Opdrachtgever'] = Project_data['Type Opdrachtgever'].astype('category')
Project_data['EMVI'] = Project_data['EMVI'].astype('category')
Project_data['Contracttype'] = Project_data['Contracttype'].astype('category')
Project_data['BKN waarde'] = Project_data['BKN waarde'].astype('category')


Project_data.dtypes

We saw only 'prekwalificatie uitslag' has NA's. This could be because not every project has a pre-qualification process before the tender. Since the type of risks are not really dependent on when this qualification takes place but might be influenced by if there is a qualification process or not, this variable will be converted to a dummy variable. When it is equal to 1 it did occur, when equal to 0 it did not occur.

The date of submitting the tender should not have much of an affect on the type of risks in a project so this will be removed too, and the same goes for 'Voormenen tot gunning'.

The start and end date could have an effect as certain seasons may deal with different risks. Another thing that could have an effect is the total time for realisation, so an extra column will be created to determine the planned project time (the differnece between start and end date.

In [ ]:
#Creating dummy variable
Project_data['prekwalificatie'] = np.where(Project_data['Prekwalificatie uitslag'].isna(), 0, 1)

#turning it into a category
Project_data['prekwalificatie'] = Project_data['prekwalificatie'].astype('category')

#dropping columns
Project_data = Project_data.drop(columns=['Indienen tender', 'Voornemen tot gunning', 'Prekwalificatie uitslag'])

In [ ]:
#creating new variable project duration

Project_data['Project duration'] = (Project_data['Eind realisatie'] - Project_data['Start realisatie']).dt.days

print(Project_data['Project duration'])

Can be seen that for one it is negative. Upon closer inspection, it seems like this is because the year has been wrongly interpreted by excel as 1930 instead of 2030. Lets change this.

In [ ]:
for i in range(len(Project_data['Project duration'])):
    if(Project_data['Project duration'][i] < 0):
        Project_data['Eind realisatie'][i] = Project_data['Eind realisatie'][i] + pd.DateOffset(years=100)

Project_data['Project duration'] = (Project_data['Eind realisatie'] - Project_data['Start realisatie']).dt.days

Now split up the datetimes to month and year so they are easier to compare. Days should not make that much of a difference as too specific for year long projects. The information is also not lost since it is still included in the 'Project Duration'

In [ ]:
Project_data['Start Year'] = Project_data['Start realisatie'].dt.year
Project_data['Start Month'] = Project_data['Start realisatie'].dt.month

Project_data['End Year'] = Project_data['Eind realisatie'].dt.year
Project_data['End Month'] = Project_data['Eind realisatie'].dt.month

Project_data = Project_data.drop(columns=['Start realisatie', 'Eind realisatie'])

In [ ]:
Project_data['Start Year'].mean()

In [ ]:
Project_data.iloc[4,4] = 'Rijkswaterstaat'

In [ ]:
from skimpy import skim

skim(Project_data.iloc[:,2:])

In [ ]:
Project_data['HLC_std'].mean()
Project_data['HLC_std'].std()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))

# Plot for Project Duration
plt.subplot(1, 2, 1)
plt.hist(Project_data['Project duration'], bins=20, edgecolor='black')
plt.xlabel('Project duration', fontsize=15)
plt.ylabel('Frequency', fontsize=17)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# Plot for Area
plt.subplot(1, 2, 2)
plt.hist(Project_data['Area'], bins=20, edgecolor='black')
plt.xlabel('Area', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# Adding (a) and (b) titles below the graphs
plt.figtext(0.25, 0.01, '(a) Histogram of Project duration', ha='center', fontsize=20)
plt.figtext(0.75, 0.01, '(b) Histogram of Area', ha='center', fontsize=20)

plt.tight_layout(rect=[0, 0.03, 1, 1])
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))

# Plot for Start Month
plt.subplot(1, 2, 1)
plt.hist(Project_data['Start Month'], bins=10, edgecolor='black')
plt.xlabel('Start Month', fontsize=15)
plt.ylabel('Frequency', fontsize=17)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# Plot for End Month
plt.subplot(1, 2, 2)
plt.hist(Project_data['End Month'], bins=10, edgecolor='black')
plt.xlabel('End Month', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# Adding (a) and (b) titles below the graphs
plt.figtext(0.25, 0.01, '(a) Histogram of Start Month', ha='center', fontsize=20)
plt.figtext(0.75, 0.01, '(b) Histogram of End Month', ha='center', fontsize=20)

plt.tight_layout(rect=[0, 0.03, 1, 1])
plt.show()


In [ ]:
plt.figure(figsize=(20, 10))

# Plot for HLC_mean
plt.subplot(1, 2, 1)
plt.hist(Project_data['HLC_mean'], bins=10, edgecolor='black')
plt.xlabel('HLC_mean', fontsize=15)
plt.ylabel('Frequency', fontsize=17)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

# Plot for HLC_std
plt.subplot(1, 2, 2)
plt.hist(Project_data['HLC_std'], bins=10, edgecolor='black')
plt.xlabel('HLC_std', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylabel('')  # Remove y-axis label

# Adding (a) and (b) titles below the graphs
plt.figtext(0.25, 0.01, '(a) Histogram of HLC_mean', ha='center', fontsize=20)
plt.figtext(0.75, 0.01, '(b) Histogram of HLC_std', ha='center', fontsize=20)

plt.tight_layout(rect=[0, 0.03, 1, 1])
plt.show()

It can be seen that there are more projects with prekwalificatie than without. Project duration has a high standard deviation due to a big right tail and the average is around 1500 days. The start year and month have less deviation than the end year and month, which makes somewhat sense as there is more control oer when to start than when to end.

The location for these 25 projects are all unique so this will not help with the modelling. The other categorical variables do have at least one category which occurs more than one since the unique observations are less than the total number of observations. This is what was expected/wanted.

Out of bag method for splitting!!!
training on all and testing on 1 for every project

What if I use the original probability instead of classification and make it a regression?

# Prepping the full dataset

In [ ]:
Clustering_data = pd.read_excel('clustering_data.xlsx', header=0)

In [ ]:
Clustering_data['Project'].unique()

In [ ]:
from fuzzywuzzy import process

# Function to find closest match
def find_closest_match(abbrev_name, full_names):
    matches = process.extractOne(abbrev_name, full_names)
    return matches[0]

# Find closest full names for each abbreviated name
closest_matches = [find_closest_match(name, Project_data['Projectnaam']) for name in Clustering_data['Project'].unique()]

print(closest_matches)


In [ ]:
#replace the project in clustering data with the closest match
Clustering_data['Project'] = Clustering_data['Project'].replace(Clustering_data['Project'].unique(), closest_matches)

In [ ]:
#Using one hot encoding for Cluster_Labels 
Cluster_labels = pd.get_dummies(Clustering_data['Cluster_Labels'], prefix='Cluster')
Label_data = pd.concat([Clustering_data['Project'], Cluster_labels], axis=1)

concat_labels = Label_data.groupby('Project').max()

In [ ]:
#merge with the project data on Projectnaam where concat_labels is index 
Project_labels = Project_data.set_index('Projectnaam')
Project_labels = Project_labels.join(concat_labels, how='inner')

Project_labels = Project_labels.reset_index()

# Modelling

In [ ]:
import sklearn

print("scikit-learn version:", sklearn.__version__)
#print("imbalanced-learn version:", imblearn.__version__)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support, make_scorer, fbeta_score, precision_score


### Prepping data for modelling

In [ ]:
#converting the target variables to a list
target_variables = Project_labels.loc[:, 'Cluster_Financieel_0':].values.tolist()

#convert to matrix
y = np.array(target_variables)

In [ ]:
#remove the target variable
X = Project_labels.loc[:, 'Locatie':'End Month']

numerical_features = X.select_dtypes(include=['int64', 'Int32', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns


In [ ]:
#turn all categorical variables into dummy variables
X_dummies = pd.get_dummies(X[categorical_features])

#remove the unecessary dummies for EMVI and prekwalificatie
X_dummies = X_dummies.drop(columns=['EMVI_Nee', 'prekwalificatie_0'])

#concatenate the numerical and dummy variables
X_transformed = pd.concat([X[numerical_features], X_dummies], axis=1)


In [ ]:
#need ot get the groups of one hot encoded variables
from collections import defaultdict

def identify_one_hot_encoded_groups(X_transformed):
    """
    Identify groups of one-hot encoded categorical features in the DataFrame.
    
    Parameters:
    - X_transformed: Features data (pandas DataFrame)
    
    Returns:
    - List of lists, where each sublist contains column names of one-hot encoded features belonging to the same categorical variable
    """
    one_hot_groups = defaultdict(list)
    
    for col in X_transformed.columns:
        if '_' in col:
            prefix = col.rsplit('_', 1)[0]
            one_hot_groups[prefix].append(col)
    
    return list(one_hot_groups.values())

In [ ]:
one_hot_groups = identify_one_hot_encoded_groups(X_transformed)

col_to_index = {col: idx for idx, col in enumerate(X_transformed.columns)} 
one_hot_group_indices = [[col_to_index[col] for col in group] for group in one_hot_groups] if one_hot_groups else []


In [ ]:
#WATCH OUT HOW TO SPLIT
#create test data from index 0, 14, 17, 19
X_test_unchanged = X_transformed.iloc[[0,14,17,19],:]
X_train_unchanged = X_transformed.drop([0,14,17,19])

# Now scale the data
scaler = StandardScaler()

X_train_scaler = scaler.fit(X_train_unchanged[numerical_features])

X_test_scaled = X_train_scaler.transform(X_test_unchanged[numerical_features])
X_test_numeric = pd.DataFrame(X_test_scaled, columns=numerical_features, index=X_test_unchanged.index)
X_test = pd.concat([X_test_numeric, X_dummies.iloc[[0,14,17,19],:]], axis=1)

X_train_scaled = X_train_scaler.transform(X_train_unchanged[numerical_features])
X_train_numeric = pd.DataFrame(X_train_scaled, columns=numerical_features, index=X_train_unchanged.index)
X_train = pd.concat([X_train_numeric, X_dummies.drop([0,14,17,19])], axis=1)

y_test = y[[0,14,17,19],:]
y_train = np.delete(y, [0,14,17,19], 0)


In [ ]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

### MLSMOTE

In [ ]:
np.where(y_train[:, 45] == 1)[0]

In [ ]:


def MLSMOTE(X, y, k=5, one_hot_group_indices=None):
    """
    Perform MLSMOTE (Multilabel Synthetic Minority Over-sampling Technique)
    
    Parameters:
    - X: Features data (numpy array)
    - y: Multi-label targets (numpy array)
    - k: Number of nearest neighbors to use for synthetic instance generation
    - one_hot_group_indices: List of lists containing indices of one-hot encoded feature columns
    
    Returns:
    - X_resampled: Resampled features
    - y_resampled: Resampled targets
    """
    if isinstance(X, pd.DataFrame):
        X = X.values

    # Ensure y is a numpy array
    y = np.array(y)

    # Calculate the initial imbalance ratio for each label
    label_counts = np.sum(y, axis=0)
    max_count = np.max(label_counts)
    
    # Nearest Neighbors model
    nn = NearestNeighbors(n_neighbors=k+1)  # +1 because the instance itself is included
    nn.fit(X)
    
    synthetic_X = []
    synthetic_y = []

    # Track the number of synthetic samples generated for each label
    synthetic_label_counts = np.copy(label_counts)

    for label in range(y.shape[1]):
        while (synthetic_label_counts[label] < max_count) and (synthetic_label_counts[label] != 0):
            # Get instances with the current label
            minority_instances = np.where(y[:, label] == 1)[0]
            
            # Randomly pick an instance from minority instances
            instance = np.random.choice(minority_instances)
            
            # Find the k-nearest neighbors
            neighbors = nn.kneighbors([X[instance]], return_distance=False).flatten()
            neighbors = neighbors[neighbors != instance]  # Exclude the instance itself
            
            # Randomly pick one neighbor
            neighbor = np.random.choice(neighbors)
            
            # Create synthetic instance
            synthetic_instance = np.zeros(X.shape[1])
            
            for group in one_hot_group_indices:
                # For one-hot encoded groups, choose one value from the group
                chosen_instance = np.random.choice([instance, neighbor])
                synthetic_instance[group] = X[chosen_instance, group]
            
            for i in range(X.shape[1]):
                if not any(i in group for group in one_hot_group_indices):
                    # For numerical features, interpolate
                    diff = X[neighbor, i] - X[instance, i]
                    gap = np.random.random()
                    synthetic_instance[i] = X[instance, i] + gap * diff
            
            # Create synthetic labelset
            synthetic_labelset = y[instance] | y[neighbor]
            

            # Append synthetic instance and labelset
            synthetic_X.append(synthetic_instance)
            synthetic_y.append(synthetic_labelset)
            
            # Update synthetic label counts
            synthetic_label_counts += synthetic_labelset

    synthetic_X = np.array(synthetic_X)
    synthetic_y = np.array(synthetic_y)
    
    # Combine original data with synthetic data
    X_resampled = np.vstack((X, synthetic_X))
    y_resampled = np.vstack((y, synthetic_y))
    
    return X_resampled, y_resampled

# Apply MLSMOTE to your training data
X_train_resampled, y_train_resampled = MLSMOTE(X_train, y_train, k=5, one_hot_group_indices=one_hot_group_indices)


retry with max extra label

In [ ]:
#compare to not using SMOTE

In [ ]:
#count the number of instances for each label
print("Number of instances for each label in the resampled training set:")
print(np.sum(y_train_resampled, axis=0))
print(np.sum(y_train, axis=0))

In [ ]:
# # #bootstrapping to get more data
# X_train_resampled, y_train_resampled = resample(X_train, y_train, replace=True, n_samples=100, random_state=0)

[![](https://mermaid.ink/img/pako:eNqdlFtv2jAUx7-K5b4GRG6M5GFSIRSoYEId28OWPbjxSeMRnMh2Vijiu9e5UbohNZofIufk_zu3HPuIo4wC9vGTIHmCNuOQhxwhWTzWhnVCJJg_Q_zA5BbNRFbkjD_V5hD_KrUIUSYgUizjaPlQW241MWdSZYJFJEVrkf3WClQ5CTIpGQipcdTrfUZjrd3c9RbBHVIZ-q51mnoBtIG9ajUTrfnGZZGD-MMkUDRJC6lA6FxaSfAuSXlOLqi-zvTXJeNbdCEpwzWZtXLgtKKqLlxrhdVGWQugrC76o2bUz_n1lix4nIkdKYG2kuklNq1MCw03EXXxlzVoGn2B59bfOY17TVzY_4nTL73W0lkdVQOrIlWsl5JHSNGKRAnjgJZABC__-UoPSvq-T2WHmi6pQwrNtKCYpal_E9uxEw8NqUS2Bf_Gtu1m33tmVCW-le__Iq2GBAtccDuSt224anWExg1ECYzi6BJ600w6aIJGEzk0jpzrmmkHP7MOfub_U-iig-P7Dx1TIhMiBDn4yK4OBkLYwDvQA8WovjyO1TBglcBOHwRfbynERA9TiEN-0lJSqOzrgUfYV6IAAxc5JQoCRvTR2mE_Jqk8W6eU6UNyNkL1uqpvqeqyMnBO-I8sewP1O_aPeI_9Ud8cuKZl2u7AHpqm-cnAB-x7fc-zXc8djryBO_Ac72Tgl8qB2R_Uy7ZHruMMreHpFZHepDo?type=png)](https://mermaid-live-editor.fly.dev/edit#pako:eNqdlFtv2jAUx7-K5b4GRG6M5GFSIRSoYEId28OWPbjxSeMRnMh2Vijiu9e5UbohNZofIufk_zu3HPuIo4wC9vGTIHmCNuOQhxwhWTzWhnVCJJg_Q_zA5BbNRFbkjD_V5hD_KrUIUSYgUizjaPlQW241MWdSZYJFJEVrkf3WClQ5CTIpGQipcdTrfUZjrd3c9RbBHVIZ-q51mnoBtIG9ajUTrfnGZZGD-MMkUDRJC6lA6FxaSfAuSXlOLqi-zvTXJeNbdCEpwzWZtXLgtKKqLlxrhdVGWQugrC76o2bUz_n1lix4nIkdKYG2kuklNq1MCw03EXXxlzVoGn2B59bfOY17TVzY_4nTL73W0lkdVQOrIlWsl5JHSNGKRAnjgJZABC__-UoPSvq-T2WHmi6pQwrNtKCYpal_E9uxEw8NqUS2Bf_Gtu1m33tmVCW-le__Iq2GBAtccDuSt224anWExg1ECYzi6BJ600w6aIJGEzk0jpzrmmkHP7MOfub_U-iig-P7Dx1TIhMiBDn4yK4OBkLYwDvQA8WovjyO1TBglcBOHwRfbynERA9TiEN-0lJSqOzrgUfYV6IAAxc5JQoCRvTR2mE_Jqk8W6eU6UNyNkL1uqpvqeqyMnBO-I8sewP1O_aPeI_9Ud8cuKZl2u7AHpqm-cnAB-x7fc-zXc8djryBO_Ac72Tgl8qB2R_Uy7ZHruMMreHpFZHepDo)

model.save

In [ ]:
X_train_HLC = X_train
X_train_noHLC = X_train.drop(columns=['HLC_mean', 'HLC_std'])
X_test_HLC = X_test
X_test_noHLC = X_test.drop(columns=['HLC_mean', 'HLC_std'])

In [ ]:
#number of labels for each test observation
y_test.sum(axis=1)

### KNN

In [ ]:

from sklearn.multioutput import MultiOutputClassifier


#tuning parameters
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15, 20],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'weights': ['uniform', 'distance']
}

# Define the F2 score as the scoring metric
f_scorer = make_scorer(fbeta_score, beta=1, average='macro', zero_division=1)

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()

Use weighted because accounts for label impabalance

zero division = 1 because it means its all predicted correctly

noHLC

In [ ]:
knn_grid_search = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
knn_grid_search.fit(X_train_noHLC, y_train)
print("Best parameters:", knn_grid_search.best_params_)
print("Best score:", knn_grid_search.best_score_)

knn_best_model_noHLC = knn_grid_search.best_estimator_
knn_y_pred_noHLC = knn_best_model_noHLC.predict(X_test_noHLC)
knn_metrics_noHLC = precision_recall_fscore_support(y_test, knn_y_pred_noHLC, average='macro', zero_division=1, beta=1)
print("Recall on test set:", knn_metrics_noHLC[1]) #0.564
print("Precision on test set:", knn_metrics_noHLC[0])#930
print("F Score on test set:", knn_metrics_noHLC[2])#0.531

HLC

In [ ]:
knn_grid_search = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
knn_grid_search.fit(X_train_HLC, y_train)
print("Best parameters:", knn_grid_search.best_params_)
print("Best score:", knn_grid_search.best_score_)

knn_best_model_HLC = knn_grid_search.best_estimator_
knn_y_pred_HLC = knn_best_model_HLC.predict(X_test_HLC)
knn_metrics_HLC = precision_recall_fscore_support(y_test, knn_y_pred_HLC, average='macro', zero_division=1, beta=1)
print("Recall on test set:", knn_metrics_HLC[1]) #0.565
print("Precision on test set:", knn_metrics_HLC[0])#0.930
print("F Score on test set:", knn_metrics_HLC[2])#0.531

Zero division = 1 makes sense for recall!

This can only happen when there are no real positive values and therefore all real ones are predicted correctly (sometimes still means that some are wrongly predicted as positive)

noHLC

In [ ]:
#predicting probability of labels
knn_proba_pred_noHLC = knn_best_model_noHLC.predict_proba(X_test_noHLC)

# Initialize a list to store the top 20 labels for each observation
top_20_indices_per_observation_knn_noHLC = []

# For each test observation, extract the top 20 labels
for i in range(X_test_noHLC.shape[0]):
    # Get the predicted probabilities for the i-th observation
    proba = np.array([knn_proba_pred_noHLC[j][i, 0] for j in range(len(knn_proba_pred_noHLC))])
    
    # Get the indices of the top 20 labels sorted by probability
    top_20_knn_indices_noHLC = np.argsort(proba)[:20][::-1]
    
    # Store the top 20 labels
    top_20_indices_per_observation_knn_noHLC.append(top_20_knn_indices_noHLC)

    top_20_knn_labels_noHLC = [concat_labels.columns[idx] for idx in top_20_knn_indices_noHLC]

    print(top_20_knn_labels_noHLC)

In [ ]:
y_pred_top_20_knn_noHLC = np.zeros((X_test_noHLC.shape[0], concat_labels.shape[1]), dtype=bool)
for i, top_20_knn_indices_noHLC in enumerate(top_20_indices_per_observation_knn_noHLC):
    y_pred_top_20_knn_noHLC[i, top_20_knn_indices_noHLC] = True

# Calculate precision for the top 20 predicted labels
precision = precision_score(y_test, y_pred_top_20_knn_noHLC, average='samples', zero_division=1)
print(f"Precision for top 20 labels: {precision}") #0.500

HLC

In [ ]:
#predicting probability of labels
knn_proba_pred_HLC = knn_best_model_HLC.predict_proba(X_test_HLC)

# Initialize a list to store the top 20 labels for each observation
top_20_indices_per_observation_knn_HLC = []

# For each test observation, extract the top 20 labels
for i in range(X_test_HLC.shape[0]):
    # Get the predicted probabilities for the i-th observation
    proba = np.array([knn_proba_pred_HLC[j][i, 0] for j in range(len(knn_proba_pred_HLC))])
    
    # Get the indices of the top 20 labels sorted by probability
    top_20_knn_indices_HLC = np.argsort(proba)[:20][::-1]
    
    # Store the top 20 labels
    top_20_indices_per_observation_knn_HLC.append(top_20_knn_indices_HLC)

    top_20_knn_labels_HLC = [concat_labels.columns[idx] for idx in top_20_knn_indices_HLC]

    print(top_20_knn_labels_HLC)

In [ ]:
y_pred_top_20_knn_HLC = np.zeros((X_test_HLC.shape[0], concat_labels.shape[1]), dtype=bool)
for i, top_20_knn_indices_HLC in enumerate(top_20_indices_per_observation_knn_HLC):
    y_pred_top_20_knn_HLC[i, top_20_knn_indices_HLC] = True

# Calculate precision for the top 20 predicted labels
precision = precision_score(y_test, y_pred_top_20_knn_HLC, average='samples', zero_division=1)
print(f"Precision for top 20 labels: {precision}") #0.500

### Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

dt_param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best'],
    'max_depth': [None, 3, 4, 5, 7],
    'min_samples_split': [2, 4, 5, 6, 7],
    'min_samples_leaf': [1, 2, 3, 4],
}

noHLC

In [ ]:
# Perform grid search with cross-validation
dt_grid_search = GridSearchCV(DecisionTreeClassifier(random_state=0), dt_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
dt_grid_search.fit(X_train_noHLC, y_train)

# Output the best parameters and score
print("Best parameters:", dt_grid_search.best_params_)
print("Best score:", dt_grid_search.best_score_)

# Get the best model
dt_best_model_noHLC = dt_grid_search.best_estimator_

# Predict the labels for the test set
dt_y_pred_noHLC = dt_best_model_noHLC.predict(X_test_noHLC)

# Evaluate the model using precision, recall, and F2 score
dt_metrics_noHLC = precision_recall_fscore_support(y_test, dt_y_pred_noHLC, average='samples', zero_division=1, beta=1)
print("Recall on test set:", dt_metrics_noHLC[1])  # 0.214
print("Precision on test set:", dt_metrics_noHLC[0])  # 0.529
print("F Score on test set:", dt_metrics_noHLC[2])  # 0.252

HLC

In [ ]:
# Perform grid search with cross-validation
dt_grid_search = GridSearchCV(DecisionTreeClassifier(random_state=0), dt_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
dt_grid_search.fit(X_train_HLC, y_train)

# Output the best parameters and score
print("Best parameters:", dt_grid_search.best_params_)
print("Best score:", dt_grid_search.best_score_)

# Get the best model
dt_best_model_HLC = dt_grid_search.best_estimator_

# Predict the labels for the test set
dt_y_pred_HLC = dt_best_model_HLC.predict(X_test_HLC)

# Evaluate the model using precision, recall, and F2 score
dt_metrics_HLC = precision_recall_fscore_support(y_test, dt_y_pred_HLC, average='macro', zero_division=1, beta=1)
print("Recall on test set:", dt_metrics_HLC[1])  # 0.536
print("Precision on test set:", dt_metrics_HLC[0])  # 0.911
print("F Score on test set:", dt_metrics_HLC[2])  # 0.507

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
import html

# Escape special characters in feature names and class names
escaped_feature_names = [html.escape(name) for name in X_train_HLC.columns]
escaped_class_names = [range(y_train.shape[1])]

# Custom export_graphviz function to exclude the 'value' parameter
def export_graphviz_custom(decision_tree, out_file=None, feature_names=None, class_names=None, filled=True, rounded=True, special_characters=True):
    from sklearn.tree import _tree

    def node_to_str(tree, node_id):
        if not isinstance(tree, _tree.Tree):
            raise ValueError('tree must be a _tree.Tree instance')
        
        if tree.children_left[node_id] == _tree.TREE_LEAF:
            return 'Leaf'
        else:
            feature = feature_names[tree.feature[node_id]]
            threshold = tree.threshold[node_id]
            return f'{feature} <= {threshold:.2f}'

    def recurse(tree, node_id, parent=None):
        name = f'node{node_id}'
        if tree.children_left[node_id] != _tree.TREE_LEAF:
            left_child = tree.children_left[node_id]
            right_child = tree.children_right[node_id]
            node_str = node_to_str(tree, node_id)
            dot.node(name, label=node_str, shape='box', style='filled', fillcolor='lightgrey')
            if parent is not None:
                dot.edge(parent, name)
            recurse(tree, left_child, parent=name)
            recurse(tree, right_child, parent=name)
        else:
            leaf_label = f'Leaf'
            dot.node(name, label=leaf_label, shape='box', style='filled', fillcolor='lightblue')
            if parent is not None:
                dot.edge(parent, name)

    dot = graphviz.Digraph()
    recurse(decision_tree.tree_, 0)
    dot.render(out_file, format='png')
    return dot

# Visualize the decision tree using the custom export_graphviz function
dot = export_graphviz_custom(dt_best_model_HLC, 
                             out_file="decision_tree", 
                             feature_names=escaped_feature_names, 
                             class_names=escaped_class_names)

# View the generated image
from IPython.display import Image
Image(filename='decision_tree.png')


In [ ]:
#number of risk labels per project
y.shape

noHLC

In [ ]:
#predicting probability of labels
dt_proba_pred_noHLC = dt_best_model_noHLC.predict_proba(X_test_noHLC)

# Initialize a list to store the top 20 labels for each observation
top_20_indices_per_observation_dt_noHLC = []

# For each test observation, extract the top 20 labels
for i in range(X_test_noHLC.shape[0]):
    # Get the predicted probabilities for the i-th observation
    proba = np.array([dt_proba_pred_noHLC[j][i, 0] for j in range(len(dt_proba_pred_noHLC))])
    
    # Get the indices of the top 20 labels sorted by probability
    top_20_dt_indices_noHLC = np.argsort(proba)[:20][::-1]
    
    # Store the top 20 labels
    top_20_indices_per_observation_dt_noHLC.append(top_20_dt_indices_noHLC)

    top_20_dt_labels_noHLC = [concat_labels.columns[idx] for idx in top_20_dt_indices_noHLC]

    print(top_20_dt_labels_noHLC)

In [ ]:
y_pred_top_20_dt_noHLC = np.zeros((X_test_noHLC.shape[0], concat_labels.shape[1]), dtype=bool)
for i, top_20_dt_indices_noHLC in enumerate(top_20_indices_per_observation_dt_noHLC):
    y_pred_top_20_dt_noHLC[i, top_20_dt_indices_noHLC] = True

# Calculate precision for the top 20 predicted labels
precision = precision_score(y_test, y_pred_top_20_dt_noHLC, average='samples', zero_division=1)
print(f"Precision for top 20 labels: {precision}") #0.525

HLC

In [ ]:
#predicting probability of labels
dt_proba_pred_HLC = dt_best_model_HLC.predict_proba(X_test_HLC)

# Initialize a list to store the top 20 labels for each observation
top_20_indices_per_observation_dt_HLC = []

# For each test observation, extract the top 20 labels
for i in range(X_test_HLC.shape[0]):
    # Get the predicted probabilities for the i-th observation
    proba = np.array([dt_proba_pred_HLC[j][i, 0] for j in range(len(dt_proba_pred_HLC))])
    
    # Get the indices of the top 20 labels sorted by probability
    top_20_dt_indices_HLC = np.argsort(proba)[:20][::-1]
    
    # Store the top 20 labels
    top_20_indices_per_observation_dt_HLC.append(top_20_dt_indices_HLC)

    top_20_labels_HLC = [concat_labels.columns[idx] for idx in top_20_dt_indices_HLC]

    print(top_20_labels_HLC)

In [ ]:
y_pred_top_20_dt_HLC = np.zeros((X_test_HLC.shape[0], concat_labels.shape[1]), dtype=bool)
for i, top_20_dt_indices_HLC in enumerate(top_20_indices_per_observation_dt_HLC):
    y_pred_top_20_dt_HLC[i, top_20_dt_indices_HLC] = True

# Calculate precision for the top 20 predicted labels
precision = precision_score(y_test, y_pred_top_20_dt_HLC, average='samples', zero_division=1)
print(f"Precision for top 20 labels: {precision}") #0.525

In [ ]:
#precision score for top 20 labels for each of the test cases
for i in range(len(y_pred_top_20_dt_HLC)):
    print(precision_score(y_test[i,:], y_pred_top_20_dt_HLC[i,:]), np.sum(y_test, axis=1)[i])

In [ ]:
np.sum(y_pred_top_20_dt_noHLC, axis=1)

In [ ]:
#which variables are the most important
# Get feature importances
dt_feature_importances = dt_best_model_HLC.feature_importances_

# Create a DataFrame to display the feature importances
dt_feature_importances_df = pd.DataFrame({'Feature': X_train_HLC.columns, 'Importance': dt_feature_importances})
dt_feature_importances_df = dt_feature_importances_df.sort_values(by='Importance', ascending=False)

# Display the top 10 most important features
print("Top 10 most important features:")
print(dt_feature_importances_df.head(10))

DT with MLSMOTE

In [ ]:
# Perform grid search with cross-validation
dt_grid_search = GridSearchCV(DecisionTreeClassifier(random_state=0), dt_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
dt_grid_search.fit(X_train_resampled, y_train_resampled)

# Output the best parameters and score
print("Best parameters:", dt_grid_search.best_params_)
print("Best score:", dt_grid_search.best_score_)

# Get the best model
dt_best_model_MLSMOTE = dt_grid_search.best_estimator_

# Predict the labels for the test set
dt_y_pred_MLSMOTE = dt_best_model_HLC.predict(X_test)

# Evaluate the model using precision, recall, and F2 score
dt_metrics_MLSMOTE = precision_recall_fscore_support(y_test, dt_y_pred_MLSMOTE, average='macro', zero_division=1, beta=1)
print("Recall on test set:", dt_metrics_MLSMOTE[1])  # 0.536
print("Precision on test set:", dt_metrics_MLSMOTE[0])  # 0.911
print("F2 Score on test set:", dt_metrics_MLSMOTE[2])  # 0.507

In [ ]:
#predicting probability of labels
dt_proba_pred_MLSMOTE = dt_best_model_MLSMOTE.predict_proba(X_test_HLC)

# Initialize a list to store the top 20 labels for each observation
top_20_indices_per_observation_dt_MLSMOTE = []

# For each test observation, extract the top 20 labels
for i in range(X_test.shape[0]):
    # Get the predicted probabilities for the i-th observation
    proba = np.array([dt_proba_pred_MLSMOTE[j][i, 0] for j in range(len(dt_proba_pred_MLSMOTE))])
    
    # Get the indices of the top 20 labels sorted by probability
    top_20_dt_indices_MLSMOTE = np.argsort(proba)[:20][::-1]
    
    # Store the top 20 labels
    top_20_indices_per_observation_dt_MLSMOTE.append(top_20_dt_indices_HLC)

    top_20_dt_labels_MLSMOTE = [concat_labels.columns[idx] for idx in top_20_dt_indices_HLC]

    print(top_20_dt_labels_MLSMOTE)

In [ ]:
y_pred_top_20_dt_MLSMOTE = np.zeros((X_test.shape[0], concat_labels.shape[1]), dtype=bool)
for i, top_20_dt_indices_MLSMOTE in enumerate(top_20_indices_per_observation_dt_MLSMOTE):
    y_pred_top_20_dt_MLSMOTE[i, top_20_dt_indices_MLSMOTE] = True

# Calculate precision for the top 20 predicted labels
precision = precision_score(y_test, y_pred_top_20_dt_MLSMOTE, average='samples', zero_division=1)
print(f"Precision for top 20 labels: {precision}") # 0.5

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {
    'n_estimators': [50],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [None, 3, 10, 15],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'ccp_alpha': [0.0, 0.1, 0.2]    
}

noHLC

In [ ]:
# Perform grid search with cross-validation
rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=0), rf_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
rf_grid_search.fit(X_train_noHLC, y_train)

# Output the best parameters and score
print("Best parameters:", rf_grid_search.best_params_)
print("Best score:", rf_grid_search.best_score_)

# Get the best model
rf_best_model_noHLC = rf_grid_search.best_estimator_

# Predict the labels for the test set
rf_y_pred_noHLC = rf_best_model_noHLC.predict(X_test_noHLC)

# Evaluate the model using precision, recall, and F2 score
rf_metrics_noHLC = precision_recall_fscore_support(y_test, rf_y_pred_noHLC, average='macro', zero_division=1, beta=1)
print("Recall on test set:", rf_metrics_noHLC[1])  # 0.550
print("Precision on test set:", rf_metrics_noHLC[0])  # 0.952
print("F2 Score on test set:", rf_metrics_noHLC[2])  # 0.516

In [ ]:
#predicting probability of labels
rf_proba_pred_noHLC = rf_best_model_noHLC.predict_proba(X_test_noHLC)

# Initialize a list to store the top 20 labels for each observation
top_20_indices_per_observation_rf_noHLC = []

# For each test observation, extract the top 20 labels
for i in range(X_test_noHLC.shape[0]):
    # Get the predicted probabilities for the i-th observation
    proba = np.array([rf_proba_pred_noHLC[j][i, 0] for j in range(len(rf_proba_pred_noHLC))])
    
    # Get the indices of the top 20 labels sorted by probability
    top_20_rf_indices_noHLC = np.argsort(proba)[:20][::-1]
    
    # Store the top 20 labels
    top_20_indices_per_observation_rf_noHLC.append(top_20_rf_indices_noHLC)

    top_20_rf_labels_noHLC = [concat_labels.columns[idx] for idx in top_20_rf_indices_noHLC]

    print(top_20_rf_labels_noHLC)

In [ ]:
y_pred_top_20_rf_noHLC = np.zeros((X_test_noHLC.shape[0], concat_labels.shape[1]), dtype=bool)
for i, top_20_rf_indices_noHLC in enumerate(top_20_indices_per_observation_rf_noHLC):
    y_pred_top_20_rf_noHLC[i, top_20_rf_indices_noHLC] = True

# Calculate precision for the top 20 predicted labels
precision = precision_score(y_test, y_pred_top_20_rf_noHLC, average='samples', zero_division=1)
print(f"Precision for top 20 labels: {precision}") #0.5125

HLC

In [ ]:
# Perform grid search with cross-validation
rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=0), rf_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
rf_grid_search.fit(X_train_HLC, y_train)

# Output the best parameters and score
print("Best parameters:", rf_grid_search.best_params_)
print("Best score:", rf_grid_search.best_score_)

# Get the best model
rf_best_model_HLC = rf_grid_search.best_estimator_

# Predict the labels for the test set
rf_y_pred_HLC = rf_best_model_HLC.predict(X_test_HLC)

# Evaluate the model using precision, recall, and F2 score
rf_metrics_HLC = precision_recall_fscore_support(y_test, rf_y_pred_HLC, average='macro', zero_division=1, beta=1)
print("Recall on test set:", rf_metrics_HLC[1])  # 0.550
print("Precision on test set:", rf_metrics_HLC[0])  # 0.952
print("F2 Score on test set:", rf_metrics_HLC[2])  # 0.516

In [ ]:
#predicting probability of labels
rf_proba_pred_HLC = rf_best_model_HLC.predict_proba(X_test_HLC)

# Initialize a list to store the top 20 labels for each observation
top_20_indices_per_observation_rf_HLC = []

# For each test observation, extract the top 20 labels
for i in range(X_test_HLC.shape[0]):
    # Get the predicted probabilities for the i-th observation
    proba = np.array([rf_proba_pred_HLC[j][i, 0] for j in range(len(rf_proba_pred_HLC))])
    
    # Get the indices of the top 20 labels sorted by probability
    top_20_rf_indices_HLC = np.argsort(proba)[:20][::-1]
    
    # Store the top 20 labels
    top_20_indices_per_observation_rf_HLC.append(top_20_rf_indices_HLC)

    top_20_rf_labels_HLC = [concat_labels.columns[idx] for idx in top_20_rf_indices_HLC]

    print(top_20_rf_labels_HLC)

In [ ]:
y_pred_top_20_rf_HLC = np.zeros((X_test_HLC.shape[0], concat_labels.shape[1]), dtype=bool)
for i, top_20_rf_indices_HLC in enumerate(top_20_indices_per_observation_rf_HLC):
    y_pred_top_20_rf_HLC[i, top_20_rf_indices_HLC] = True

# Calculate precision for the top 20 predicted labels
precision = precision_score(y_test, y_pred_top_20_rf_HLC, average='samples', zero_division=1)
print(f"Precision for top 20 labels: {precision}") #0.5125

In [ ]:
#most important features
# Get feature importances
rf_feature_importances = rf_best_model_HLC.feature_importances_

# Create a DataFrame to display the feature importances
rf_feature_importances_df = pd.DataFrame({'Feature': X_train_HLC.columns, 'Importance': rf_feature_importances})
rf_feature_importances_df = rf_feature_importances_df.sort_values(by='Importance', ascending=False)

# Display the top 10 most important features
print("Top 10 most important features:")
print(rf_feature_importances_df.head(10))

### Save best model

In [ ]:
#run decision tree with all X
scaler = StandardScaler()
X_full = scaler.fit_transform(X_transformed[numerical_features])
X_full_numeric = pd.DataFrame(X_full, columns=numerical_features, index=X_transformed.index)
X_final = pd.concat([X_full_numeric, X_dummies], axis=1)

# Perform grid search with cross-validation
dt_grid_search_final = GridSearchCV(DecisionTreeClassifier(random_state=0), dt_param_grid, cv=loo, scoring=f_scorer, n_jobs=-2)
dt_grid_search_final.fit(X_final, y)

# select the best model
dt_best_final_model = dt_grid_search_final.best_estimator_


In [ ]:
#save the best model in a pickle file
import pickle

# Save the best model to a file
with open('best_final_model.pkl', 'wb') as file:
    pickle.dump(dt_best_final_model, file)